# Google - Cloud Service Customer Retention and Cost Efficiency

```SQL
CREATE TABLE fct_transactions (
    transaction_id INTEGER,
    customer_id INTEGER,
    service_tier_code VARCHAR,
    transaction_date DATE
);

INSERT INTO fct_transactions (transaction_id, customer_id, service_tier_code, transaction_date)
VALUES
    (1, 101, 'PREM_BASIC', '2024-04-05'),
    (2, 102, 'PREM_PLUS', '2024-04-10'),
    (3, 103, 'BASIC', '2024-04-15'),
    (4, 101, 'PREM_BASIC', '2024-04-20'),
    (5, 104, 'PREM_ULTIMATE', '2024-04-25'),
    (6, 105, 'STANDARD', '2024-05-02'),
    (7, 106, 'BASIC', '2024-05-05'),
    (8, 107, 'PREM_PLUS', '2024-05-08'),
    (9, 105, 'STANDARD', '2024-05-12'),
    (10, 108, 'PREM_BASIC', '2024-05-15'),
    (11, 109, 'STANDARD', '2024-05-20'),
    (12, 105, 'PREM_BASIC', '2024-06-03'),
    (13, 111, 'PREM_BASIC', '2024-06-05'),
    (14, 112, 'STANDARD', '2024-06-07'),
    (15, 113, 'PREM_PLUS', '2024-06-10'),
    (16, 111, 'STANDARD', '2024-06-13'),
    (17, 114, 'PREM_PLUS', '2024-06-16'),
    (18, 115, 'PREM_ULTIMATE', '2024-06-20'),
    (19, 116, 'STANDARD', '2024-06-23'),
    (20, 117, 'PREM_BASIC', '2024-06-28');
```

In [1]:
import pandas as pd   
import numpy as np   

In [2]:
df_transaction = pd.read_csv('Data/007/fct_transactions_2.csv')
df_transaction.head()

,transaction_id,customer_id,service_tier_code,transaction_date
0,1,101,PREM_BASIC,2024-04-05
1,2,102,PREM_PLUS,2024-04-10
2,3,103,BASIC,2024-04-15
3,4,101,PREM_BASIC,2024-04-20
4,5,104,PREM_ULTIMATE,2024-04-25


# Pregunta 1

### Queremos evaluar la adopción temprana del servicio premium entre los clientes empresariales. ¿Cuántos clientes únicos, cuyos códigos de nivel de servicio comienzan con 'PREM', completaron transacciones desde el 1 de abril hasta el 30 de abril de 2024?

In [4]:
df_abril = df_transaction[
    (df_transaction['transaction_date'].between('2024-04-01','2024-04-30')) &
    (df_transaction['service_tier_code'].str.startswith('PREM'))
    ]


total_clientes = df_abril['customer_id'].nunique()

total_clientes

3

```SQL
SELECT
    COUNT(distinct customer_id) AS unique_premium_customers
FROM fct_transactions
WHERE service_tier_code like 'PREM%'
AND transaction_date BETWEEN '2024-04-01' AND '2024-04-30';

# Pregunta 2

### Necesitamos entender las tendencias de uso de los diferentes niveles de servicio para perfeccionar nuestros paquetes de servicios. ¿Qué códigos de nivel de servicio fueron utilizados con mayor frecuencia por los clientes para realizar transacciones en mayo de 2024, ordenados de mayor a menor frecuencia?

In [7]:
df_mayo = df_transaction[
    df_transaction['transaction_date'].between('2024-05-01','2024-05-31')
]

df_service_count = df_mayo.groupby('service_tier_code')['transaction_id'].count().sort_values(ascending=False)

df_service_count

service_tier_code
STANDARD      3
BASIC         1
PREM_BASIC    1
PREM_PLUS     1
Name: transaction_id, dtype: int64

```SQL
SELECT
    service_tier_code,
    count(transaction_id) AS frequency
FROM fct_transactions
WHERE EXTRACT (YEAR FROM transaction_date) = 2024
AND EXTRACT (MONTH FROM transaction_date) = 5
GROUP BY service_tier_code
ORDER BY  frequency DESC;
```

# Pregunta 3

### Queremos identificar con precisión los niveles de servicio más activos para informar los ajustes de precios en nuestras ofertas de la nube para empresas. Para las transacciones realizadas entre el 1 y el 30 de junio de 2024, ¿cuáles son los tres niveles de servicio principales según el volumen de transacciones y cuántas transacciones se registraron para cada uno?

In [10]:
df_junio = df_transaction[df_transaction['transaction_date'].between('2024-06-01','2024-06-30')]

df_service = df_junio.groupby('service_tier_code')['transaction_id'].count().sort_values(ascending=False).head(3)

df_service

service_tier_code
PREM_BASIC    3
STANDARD      3
PREM_PLUS     2
Name: transaction_id, dtype: int64

```SQL
SELECT
    service_tier_code,
    count(transaction_id) AS total_transaction
FROM fct_transactions
WHERE transaction_date BETWEEN '2024-06-01' AND '2024-06-30'
GROUP BY service_tier_code
ORDER BY total_transaction DESC
LIMIT 3;
```